In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.parse import urlparse
import pandas as pd
import os
import json
from src.Web_Scraping.basic_functions import (is_scraping_allowed, remove_cookie_banners, getWebsiteText, getWebsiteText_v2, format_string, remove_sentence_with_keyword)

In [6]:
# path to csv file
data = pd.read_csv("/content/drive/MyDrive/Startup_Scraping/startup_list_0_5000_prep.csv", index_col=0)
data.head()

Check the robots.txt of each web page, by adjusting range(1751, 3500)
-> all data: len(data)

In [ ]:
for i in range(1751, 3500):
  print(i) # only progress information
  url = data['website_url'][i]
  result = is_scraping_allowed(url)
  data.loc[i, "check_robots"] = result

In [ ]:
#Save data as new csv
data.to_csv("/content/drive/MyDrive/")

Gets Text from the allowed websites and saves it as txt

In [ ]:
#path for the text files

path = "./txt_files_3501_5000/"
os.makedirs(path, exist_ok=True)

In [ ]:
for i in range(0, 1501):
  if(data["check_robots"][i]==True):
    url = data['website_url'][i]
    text = getWebsiteText(url)
    lines = text.splitlines()
    new_lines = [line for line in lines if line]
    new_text = "\n".join(new_lines)
    # lowercase + remove everything in name but chars
    # because some names contains symbols e.g. ":" -> would destroy a textfile
    name = data["name"][i].lower()
    name = re.sub(r"[^a-z]", "", name)
    print(f"{i}: {name}") # only progress information
    with open(path+name+".txt", "w", encoding="utf-8") as f:
      f.write(new_text)


Gets Text from the allowed websites and saves as one json file

In [65]:
data = pd.read_csv("/content/drive/MyDrive/Startup_Scraping/startup_list_0_5000_prep.csv", index_col=0)
data['original_idx'] = data.index
data = data.loc[data["check_robots"] == True]

In [ ]:
data_dict = {}
# Iterate over the rows of the DataFrame
for index, row in data.iterrows():
    print(index)
    name = row['name']
    id = row['original_idx']
    url = row['website_url']

    text = getWebsiteText_v2(url)

    # Remove
    lines = text.splitlines()
    new_lines = [line for line in lines if line]
    new_text = "\n".join(new_lines)
    website_text = format_string(new_text)


    # Create a dictionary for the current row and add it to the data_dict
    row_dict = {
        'name': name,
        'original_idx': id,
        'website_url': url,
        'website_text': website_text
    }
    data_dict[index] = row_dict


json_object = json.dumps(data_dict, indent=4, ensure_ascii=False)
with open("/content/drive/MyDrive/Startup_Scraping/startup_list_0_5000_without_cookie.json", "w", encoding='utf-8') as outfile:
    outfile.write(json_object)

In [29]:
json_object = json.dumps(data_dict, indent=4, ensure_ascii=False)
with open("/content/drive/MyDrive/Startup_Scraping/startup_list_0_5000_without_cookie.json", "w", encoding='utf-8') as outfile:
    outfile.write(json_object)